In [39]:
import sys
import json
sys.path.insert(0, r'../../../quetzal')
from pathlib import Path

import zipfile
import os
import time
import geopandas as gpd
import pandas as pd
import numpy as np
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from shapely.geometry import Point, LineString
from syspy.spatial.spatial import add_geometry_coordinates, nearest
from sklearn.neighbors import NearestNeighbors
from typing import Literal
from numba import jit, njit
import numba as nb
#num_cores = 1
print('numba threads',nb.config.NUMBA_NUM_THREADS)

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
io_engine = 'pyogrio' if on_lambda else 'fiona'

numba threads 8


In [42]:
general = {'step_size':'0.0025'}
params = {'general':general}
default = {'training_folder': '../../scenarios/alula', 'params':params} # Default execution parametersmanual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))

manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
argv

{'training_folder': '../../scenarios/alula',
 'params': {'general': {'step_size': '0.0025'}}}

In [43]:
# from quetzal_cyclops
def get_epsg(lat: float, lon: float) -> int:
    '''
    lat, lon or y, x
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

# from quetzal_cyclops
def zones_nearest_node(zones,nodes,drop_duplicates=False):
    # getting zones centroids
    centroid = zones.copy()
    centroid['geometry'] = centroid.centroid
    # finding nearest node
    neigh = nearest(centroid, nodes, n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    centroid['node_index'] = centroid.index.map(zone_node_dict.get)
    #print('max_distance found: ', neigh['distance'].max())
    # check for duplicated nodes. if there is. drop the duplicated zones.
    if drop_duplicates:
        if len(centroid.drop_duplicates('node_index')) != len(centroid):
            print('there is zones associates to the same road_node')
            # duplicated = centroid[centroid['node_index'].duplicated()]['node_index'].values
            print('dropping zones: ')
            print(centroid[centroid['node_index'].duplicated()].index.values)
            centroid = centroid.drop_duplicates('node_index')
    return centroid

In [44]:
from quetzal.engine.pathfinder_utils import simple_routing, sparse_matrix, get_path
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra

@jit(nopython=True)
def _unstack(mat):
    # return non inf values in mat as [[row,col,val],[row,col,val]]. so, [o,d,val].
    # pd.DataFrame of this gives us [origin, destination, value] as columns
    row, col = np.where(np.isfinite(mat))
    res = np.zeros((len(row),3))
    for it in nb.prange(len(col)):
        i=row[it]
        j=col[it]
        d=mat[i,j]
        res[it]=[i,j,d]
    return res

def routing(origin, destination, links, weight_col='time', dijkstra_limit=np.inf):
    mat, node_index = sparse_matrix(links[['a', 'b', weight_col]].values)
    index_node = {v: k for k, v in node_index.items()}
    # liste des origines pour le dijkstra
    origin_sparse = [node_index[x] for x in origin]
    origin_dict =  {i:val for i,val in enumerate(origin_sparse)}
    # list des destinations 
    destination_sparse = [node_index[x] for x in destination]
    destination_dict =  {i:val for i,val in enumerate(destination_sparse)}
    # dijktra on the road network from node = incices to every other nodes.
    # from b to a.
    dist_matrix = dijkstra(
        csgraph=mat,
        directed=True,
        indices=origin_sparse,
        return_predecessors=False,
        limit=dijkstra_limit
    )
    # remove non-used destination
    dist_matrix = dist_matrix[:,destination_sparse]
    # unstack amtrix
    dist_matrix = pd.DataFrame(_unstack(dist_matrix),columns=['origin', 'destination', weight_col])
    # rename origin and destination with original indexes.
    dist_matrix['origin'] = dist_matrix['origin'].apply(lambda x: index_node.get(origin_dict.get(x)))
    dist_matrix['destination'] = dist_matrix['destination'].apply(lambda x: index_node.get(destination_dict.get(x)))
    return dist_matrix

In [45]:
def get_catchment_dist(link: gpd.GeoDataFrame, catchment_radius: dict, default: float=500):
    route_type = link['route_type'].unique()
    if len(route_type)>1:
        print('multiple route type for a single route_id.. using first one for catchment radius')
    route_type = route_type[0]
    return catchment_radius.get(route_type, default)


def nearest_radius(one, many, radius=100,to_crs=None):
    try:
        # Assert df_many.index.is_unique
        assert one.index.is_unique
        assert many.index.is_unique
    except AssertionError:
        msg = 'Index of one or many should not contain duplicates'
        print(msg)
        warnings.warn(msg)
    many = add_geometry_coordinates(many, columns=['x_geometry', 'y_geometry'], to_crs=to_crs)
    one = add_geometry_coordinates(one, columns=['x_geometry', 'y_geometry'], to_crs=to_crs)
    
    x = many[['x_geometry', 'y_geometry']].values
    # Fit Nearest neighbors model
    #nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(x)
    nbrs = NearestNeighbors(radius=radius,algorithm='ball_tree').fit(x)

    y = one[['x_geometry', 'y_geometry']].values

    #distances, indices = nbrs.kneighbors(y,return_distance=True)
    distances, indices = nbrs.radius_neighbors(y, radius = radius, return_distance=True)

    indices = pd.DataFrame(indices)
    indices = pd.DataFrame(indices.stack(), columns=['index_nn']).reset_index().rename(
        columns={'level_0': 'ix_one', 'level_1': 'rank'}
    )
    indices['distances'] = distances
    return indices

def create_mesh(zones: gpd.GeoDataFrame ,step: float = 0.01) -> gpd.GeoDataFrame:
    '''
    create a mesh in the zones total bbox at every step (in the units of the zones crs)
    step: degree if crs=4326, else meters. 0.01 deg ~ 1km
    '''
    x_max, y_max = zones.bounds.max()[['maxx','maxy']].values
    x_min, y_min = zones.bounds.min()[['minx','miny']].values

    points = []
    x = x_min
    while x<x_max:
        y = y_min
        while y<y_max:
            points.append(Point(x,y))
            y += step
        x += step
    points = gpd.GeoDataFrame(geometry=points,crs=zones.crs)
    points.index.name='index'
    return points

# https://stackoverflow.com/questions/36399381/whats-the-fastest-way-of-checking-if-a-point-is-inside-a-polygon-in-python

@jit(nopython=True)
def fast_point_in_polygon(x: float, y: float , poly: np.ndarray) -> bool:
    n = len(poly)
    inside = False
    p2x = 0.0
    p2y = 0.0
    xints = 0.0
    p1x,p1y = poly[0]
    for i in nb.prange(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y
        
    return inside


@njit(parallel=True)
def fast_points_in_polygon(points:np.ndarray, polygon:np.ndarray) -> np.ndarray:
    D = np.empty(len(points), dtype=nb.boolean) 
    for i in nb.prange(0, len(points)):
        D[i] = fast_point_in_polygon(points[i,0], points[i,1], polygon)
    return np.where(D)[0]

def points_in_polygon(points:np.ndarray, polygon:gpd.GeoDataFrame) -> np.ndarray:
    '''
    return a list of point in the polygon. values are the index in the points array.
    
    points:np.array[np.array[float,float]]
        list of all the points coords (x,y)
    polygon: gpd.GeoDataFrame
        geodataframe of multiples polygons.
    '''
    try:
        poly = np.array([*polygon.exterior.coords])
        return fast_points_in_polygon(points,poly)
    except:
        res=np.array([])
        #polygon = polygon.geoms
        for i in range(len(polygon)):
            poly = np.array([*polygon[i].exterior.coords])
            val =fast_points_in_polygon(points,poly)
            res = np.append(res,val)
        return res

def population_to_mesh(population: gpd.GeoDataFrame,
                       mesh: gpd.GeoDataFrame = None,
                       step: float = 0.01,
                       col: str = 'population', 
                       fill_missing: Literal['centroid', 'nearest', None] = 'centroid') ->  gpd.GeoDataFrame:
    '''
    create a mesh in the zones total bbox at every step (in the units of the zones crs)
    and assign the population to each node equaly (if 2 node in a zone. they have each 50% of the population)
    population:
        geodataframe with total population by zones ans zones geomerty
     mesh:
        road nodes for example. if None. it will be created with equal step (variable step.)
    step: 
        if mesh is None, Distance between each point degree if crs=4326, else meters. 0.01 deg ~ 1km
    col:
        column name with data to aggregation (population)
    fill_missing: 'centroid', 'nearest', or None
        centroid: zones centroid with no mesh node inside will be added to the mesh
        nearest: zones population with no mesh point inside will be added to the nearest mesh point.
    '''
    import warnings
    warnings.filterwarnings('ignore')
    population=population.copy()
    if population.index.name != 'index':
        population.index.name = 'index'
    # use existing mesh (points .geosjon) or create one.
    if mesh is not None:
        # we need numerical indexes. also,
        # new nodes will be added (new index) for zones with no points inside.
        points = mesh.copy()
        points = points.reset_index(names='node_index')
        points.index.name='index'
    else:
        points = create_mesh(population, step=step)
        
    points_coords = np.array([point.coords[0] for point in points['geometry'].values])
    
    population['nodes'] = population['geometry'].apply(lambda x: points_in_polygon(points_coords,x))
    
    nodes = population.reset_index()[['index','nodes',col]].copy()
    nodes = nodes.explode('nodes').dropna()
    print(len(nodes[nodes['nodes'].duplicated()]),'nodes in multiple zones. will be match to a single zone.')
    
    
    zone_index_dict = nodes.set_index('nodes')['index'].to_dict()
    points['zone'] = points.index.map(zone_index_dict)

    pop_dict = nodes.set_index('nodes')[col].to_dict()
    points[col] = points.index.map(pop_dict)
    points = points.dropna()
    
    # get number of points per zones. divide population equaly between each points
    len_dict = points.groupby('zone')[col].agg(len).to_dict()
    points['num_points'] = points['zone'].apply(lambda x:len_dict.get(x))
    points[col] = points[col] / points['num_points']
    points = points.drop(columns = ['num_points'])
    
    print(len(population) - len(points['zone'].unique()),'unfounded zones')
    
    zones_list = points['zone'].unique()
    unfounded_zones = population.loc[~population.index.isin(zones_list)][['geometry',col]]
    if fill_missing == 'centroid':
        print('Unfound zones centroid will be added to mesh')
        # append unfounded zones centroids as in mesh
        unfounded_zones['geometry'] = unfounded_zones.centroid
        unfounded_zones = unfounded_zones.reset_index().rename(columns={'index':'zone'})
        points = pd.concat([points,unfounded_zones]).reset_index(drop=True)
        points.index.name='index'
    elif fill_missing == 'nearest':
        print('unfound zone will be added to nearest mesh node. zone_index will be lost')
        unfounded_zones = zones_nearest_node(unfounded_zones,points)
        pop_to_append = unfounded_zones.groupby('node_index')[[col]].sum()

        points = points.merge(pop_to_append,left_index=True,right_index=True,how='left')
        points[col+'_y'] = points[col+'_y'].fillna(0)

        points[col] = points[col+'_x'] + points[col+'_y']
        points = points.drop(columns=[col+'_x', col+'_y'])
    else:
        pass
    
    
    points.index.name='index'
    
    return points

In [46]:
def get_acf_distances(nodes: gpd.GeoDataFrame, 
                      mesh: gpd.GeoDataFrame, 
                      crs:int,
                      max_dist: float = 3000) -> gpd.GeoDataFrame:
    '''
    with nearest kneibor in a radius.
    for pt node in nodes, get all mesh nodes in a distance < max_dist
    
    return gpd.Geodateframe with [node_index, mesh_index, distances, population]
    '''

    node_dist = nearest_radius(nodes, mesh, radius=max_dist, to_crs=crs)
    node_dist = node_dist.rename(columns={'ix_one': 'node_index','index_nn':'mesh_index'}).drop(columns='rank')

    nodes_index_dict = nodes.reset_index()['index'].to_dict()
    node_dist['node_index'] = node_dist['node_index'].apply(lambda x: nodes_index_dict.get(x))

    node_dist = node_dist.explode(['mesh_index','distances'])
    population_dict = mesh['population'].to_dict()
    node_dist['population'] = node_dist['mesh_index'].apply(lambda x: population_dict.get(x))
    return node_dist

def get_routing_distances(nodes: gpd.GeoDataFrame, 
                         rnodes: gpd.GeoDataFrame, 
                         rlinks: gpd.GeoDataFrame, 
                         mesh: gpd.GeoDataFrame, 
                         weight_col:str = 'length', 
                         dijkstra_limit: float = np.inf) -> gpd.GeoDataFrame:
    '''
    with dijktra on road network.
    for pt node in nodes, get all mesh nodes in a distance < max_dist. can be change with weight_col
    ex: weight_col = 'time', and dijkstra_limit = 120secs
    
    return gpd.Geodateframe with [node_index, mesh_index, distances, population]
    '''

    # transform PT nodes to nearest road nodes
    node_to_rnode_df = zones_nearest_node(nodes,rnodes)[['node_index']]

    node_rnodes_dict = node_to_rnode_df['node_index'].to_dict()
    rnodes_node_dict = node_to_rnode_df.reset_index().groupby('node_index').agg(list)['index'].to_dict()

    # there may be multiples nodes pointing to the same rnode. so rnodes_node_dict values are lists.
    # need to added them back at the end when we go from rnode to nodes
    origins = list(set(node_rnodes_dict.values()))
    destinations = mesh['node_index'].values
    mat = routing(origins, destinations, rlinks, weight_col=weight_col, dijkstra_limit=dijkstra_limit)

    mat = mat.merge(mesh.reset_index()[['index','node_index','population']],left_on='destination',right_on='node_index',how='left')
    mat = mat.drop(columns=['destination','node_index']).rename(columns={'index':'mesh_index'})

    mat['origin'] = mat['origin'].apply(lambda x: rnodes_node_dict.get(x))
    mat = mat.explode('origin')
    mat = mat.rename(columns={'origin':'node_index', weight_col:'distances'})
    return mat

In [47]:
def read_quenedi_zip(zip_path, foldername=''):
    archive = zipfile.ZipFile(zip_path, 'r')
    link_data = archive.read(foldername + 'links.geojson')
    node_data = archive.read(foldername + 'nodes.geojson')
    l_ = json.loads(link_data)
    n_ = json.loads(node_data)
    links = gpd.read_file(json.dumps(l_)).set_index('index')
    nodes = gpd.read_file(json.dumps(n_)).set_index('index')

    return links, nodes

# Folders stucture and params

everything is on S3 (nothing on ECR) so no direct input folder. just scenarios/{scen}/inputs/

Data folder structure

In [48]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')

Inputs

In [59]:
argv['params']['general'].get('step_size')
step_size =  argv['params']['general'].get('step_size')
step_size = float(step_size)
step_size

0.0025

In [60]:
links = gpd.read_file(pt_folder + 'links.geojson', engine=io_engine) 
nodes = gpd.read_file(pt_folder + 'nodes.geojson', engine=io_engine)
links = links.set_index('index')
nodes = nodes.set_index('index')

In [61]:
capacity = dict(zip(links['route_type'], links['capacity']))
catchment_radius = dict(zip(links['route_type'], links['catchment_radius']))
catchment_radius = {k:float(v) for k,v in catchment_radius.items()}
default_catchment_radius = 500

params = {
    'catchment_radius': catchment_radius, 
    'capacity': capacity
          }

default['params'] = params

In [62]:
population_file = os.path.join(input_folder, 'population.geojson')
population_file_provided = os.path.isfile(population_file)
if population_file_provided :
    population = gpd.read_file(population_file, engine=io_engine)
    if 'index' in population.columns:
        population = population.set_index('index')
    else:
        population.index.name='index'
    assert 'density' in population.columns, 'need density column. in km2'
    assert population.crs == 4326, 'population.geojson CRS must be EPSG:4326'
print('population?',population_file_provided)

population? True


In [63]:
jobs_file = os.path.join(input_folder, 'jobs.geojson')
jobs_file_provided = os.path.isfile(jobs_file)
if jobs_file_provided :
    jobs = gpd.read_file(jobs_file, engine=io_engine)
    if 'index' in jobs.columns:
        jobs = jobs.set_index('index')
    else:
        jobs.index.name='index'
    assert 'density' in jobs.columns, 'need density column. in km2'
    assert jobs.crs == 4326, 'population.geojson CRS must be EPSG:4326'
print('jobs?',jobs_file_provided)

jobs? True


In [64]:
tourists_file = os.path.join(input_folder, 'tourists.geojson')
tourists_file_provided = os.path.isfile(tourists_file)
if tourists_file_provided :
    tourists = gpd.read_file(tourists_file, engine=io_engine)
    if 'index' in tourists.columns:
        tourists = tourists.set_index('index')
    else:
        tourists.index.name='index'
    assert 'density' in tourists.columns, 'need density column. in km2'
    assert tourists.crs == 4326, 'population.geojson CRS must be EPSG:4326'
print('tourists?',tourists_file_provided)

tourists? True


In [65]:
rnodes_file = os.path.join(road_folder, 'road_nodes.geojson')
rnodes_file_provided = os.path.isfile(rnodes_file)
if rnodes_file_provided:
    rnodes = gpd.read_file(os.path.join(road_folder, 'road_nodes.geojson'), engine=io_engine)
    rnodes = rnodes.set_index('index')
    rlinks = gpd.read_file(os.path.join(road_folder, 'road_links.geojson'), engine=io_engine)
    rlinks = rlinks.set_index('index')
print('rnodes?',rnodes_file_provided)

rnodes? False


In [66]:
od_file = os.path.join(od_folder, 'od.geojson')
od_file_provided = os.path.isfile(od_file)
if od_file_provided:
    od_test = gpd.read_file(od_file, engine=io_engine)
    if 'name' not in od_test.columns:
        od_test['name'] = od_test['index']
    od_test['name'] = od_test['name'].fillna(od_test['index'].astype(str))
print('od?',od_file_provided)

od? False


# People catchment

In [67]:
# find meters CRS
centroid = [*LineString(nodes.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])
crs

/tmp/ipykernel_331324/550055817.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = [*LineString(nodes.centroid.values).centroid.coords][0]


32637

In [68]:
max_dist = max(max(catchment_radius.values()), default_catchment_radius)
max_dist

500.0

In [69]:
print('num route_id:', len(links['route_id'].unique()))
print('num route_type:', len(links['route_type'].unique()))

num route_id: 19
num route_type: 3


In [70]:
#init results dfs
df_route_id = pd.DataFrame(index=links['route_id'].unique())
df_route_id.index.name = 'route_id'
df_route_id = df_route_id.reset_index()
df_route_id = df_route_id.merge(links[['route_id', 'route_type', 'capacity', 'headway']], on='route_id', how='left')
df_route_id = df_route_id.rename(columns={'capacity': 'veh_capacity (PAX)'})
df_route_id = df_route_id.drop_duplicates()
df_route_id = df_route_id.set_index('route_id')

df_route_type = pd.DataFrame(index=links['route_type'].unique())
df_route_type.index.name='route_type'

In [71]:
def get_catchment(col='route_id', node_dist=None):
    #get all nodes with col filter
    link = links.groupby(col)[['a', 'b', 'route_type']].agg({'a': set, 'b': set, 'route_type':'first'})
    link['node'] = link.apply(lambda row: row['a'].union(row['b']), axis=1)
    link = link.drop(columns=['a', 'b'])
    # add catchment radius for the route_type
    link['catchment_radius'] = link['route_type'].apply(lambda x: catchment_radius.get(x, default_catchment_radius))

    col_exist = col == 'route_type' # cannot explode if index == route_type (a column)
    link = link.explode('node').reset_index(drop=col_exist)
    link = node_dist.merge(link, left_on='node_index', right_on='node')
    #filter by distance
    link = link[link['distances'] <= link['catchment_radius']]
    #drop duplicated mesh nodes (we count only one time)
    link = link.drop_duplicates(subset=['mesh_index', col], keep='first')

    return link.groupby(col)['population'].sum().to_dict()

## 1. Population

### Preparation

In [72]:
if population_file_provided : 
    population['area (km2)'] = population.to_crs(crs).area * 1e-6
    print(population['area (km2)'].sum())

1046.2109408207182


In [73]:
if population_file_provided :
    population['population'] = population['density'] * population['area (km2)']
    print(population['population'].sum())

162117.16124220772


### Mesh

In [75]:
if not population_file_provided:
    mesh_pop = gpd.GeoDataFrame(index=[0],data={'zone':'centroid','population':0},geometry=[Point(centroid[0],centroid[1])])
    mesh_pop.index.name = 'index'
    mesh_pop.crs=4326
    if rnodes_file_provided:
        mesh_pop['node_index'] = rnodes.index[0]
elif rnodes_file_provided:
    # use rnodes as mesh.
    print('using road_nodes')
    mesh_pop = population_to_mesh(population, mesh=rnodes, step=step_size, col='population', fill_missing='nearest')
else:
    # create a mesh
    #0.01 = 1km 0.005 = 500m
    mesh_pop = population_to_mesh(population, step=step_size, col = 'population', fill_missing='centroid')

38 nodes in multiple zones. will be match to a single zone.
3 unfounded zones
Unfound zones centroid will be added to mesh


In [76]:
mesh_pop.to_file(output_folder + 'population_mesh.geojson',driver='GeoJSON',engine=io_engine)

### Catchment

In [77]:
# find TC nodes to mesh distance

In [78]:
if rnodes_file_provided:
    print('using road_nodes')
    node_dist_pop = get_routing_distances(nodes, rnodes, rlinks, mesh_pop, 'length', max_dist)
else:
    node_dist_pop = get_acf_distances(nodes, mesh_pop, crs, max_dist)

### Metrics

In [79]:
res_route_pop = get_catchment('route_id', node_dist=node_dist_pop)

df_route_id['catchment population'] = res_route_pop
print(sum([item for key,item in res_route_pop.items()]))

88906.13412295088


In [80]:
res_mode_pop = get_catchment('route_type', node_dist=node_dist_pop)

df_route_type['catchment population'] = res_mode_pop
print(sum([item for key,item in res_mode_pop.items()]))

65704.22645771733


In [81]:
if False:
    links['network']=True
    res=[]
    dists = [0,1,10,20,50,100,250,500,800,1000]
    for dist in dists:
        col='network'
        link = links.groupby(col)[['a','b','route_type']].agg({'a':set,'b':set,'route_type':'first'})
        link['node'] = link.apply(lambda row: row['a'].union(row['b']), axis=1)
        link = link.drop(columns=['a','b'])
        # add catchment radius for the route_type
        link['catchment_radius'] =dist

        col_exist = col == 'route_type' # cannot explode if index == route_type (a column)
        link = link.explode('node').reset_index(drop=col_exist)
        link = node_dist.merge(link, left_on='node_index', right_on='node')
        #filter by distance
        link = link[link['distances'] <= link['catchment_radius']]
        #drop duplicated mesh nodes (we count only one time)
        link = link.drop_duplicates(subset=['mesh_index',col],keep='first')
        volume = link['population'].sum()
        res.append(volume)
    plt.plot(dists,res)

## 2. Jobs

### Preparation

In [82]:
if jobs_file_provided : 
    jobs['area (km2)'] = jobs.to_crs(crs).area * 1e-6
    print(jobs['area (km2)'].sum())

1046.2109408207182


In [83]:
if jobs_file_provided :
    jobs['population'] = jobs['density'] * jobs['area (km2)']
    print(jobs['population'].sum())

100377.43417597521


### Mesh

In [85]:
if not jobs_file_provided:
    mesh_jobs = gpd.GeoDataFrame(index=[0],data={'zone':'centroid','population':0},geometry=[Point(centroid[0],centroid[1])])
    mesh_jobs.index.name = 'index'
    mesh_jobs.crs=4326
    if rnodes_file_provided:
        mesh_jobs['node_index'] = rnodes.index[0]
elif rnodes_file_provided:
    # use rnodes as mesh.
    print('using road_nodes')
    mesh_jobs = population_to_mesh(jobs, mesh=rnodes, step=step_size, col='population', fill_missing='nearest')
else:
    # create a mesh
    #0.01 = 1km 0.005 = 500m
    mesh_jobs = population_to_mesh(jobs, step=step_size, col='population', fill_missing='centroid')

38 nodes in multiple zones. will be match to a single zone.
3 unfounded zones
Unfound zones centroid will be added to mesh


In [86]:
mesh_jobs.to_file(output_folder + 'jobs_mesh.geojson', driver='GeoJSON', engine=io_engine)

### Catchment

In [87]:
# find TC nodes to mesh distance

In [88]:
if rnodes_file_provided:
    print('using road_nodes')
    node_dist_jobs = get_routing_distances(nodes, rnodes, rlinks, mesh_jobs, 'length', max_dist)
else:
    node_dist_jobs = get_acf_distances(nodes, mesh_jobs, crs, max_dist)

### Metrics

In [89]:
res_route_jobs = get_catchment('route_id', node_dist=node_dist_jobs)

df_route_id['catchment jobs'] = res_route_jobs
print(sum([item for key,item in res_route_jobs.items()]))

68815.21138011852


In [90]:
res_mode_jobs = get_catchment('route_type', node_dist=node_dist_jobs)

df_route_type['catchment jobs'] = res_mode_jobs
print(sum([item for key,item in res_route_jobs.items()]))

68815.21138011852


## 3. Tourists

### Preparation

In [91]:
if tourists_file_provided : 
    tourists['area (km2)'] = tourists.to_crs(crs).area * 1e-6
    print(tourists['area (km2)'].sum())

1046.2109408207182


In [92]:
if tourists_file_provided :
    tourists['population'] = tourists['density'] * tourists['area (km2)']
    print(tourists['population'].sum())

73942.85116376789


### Mesh

In [93]:
if not tourists_file_provided:
    mesh_tour = gpd.GeoDataFrame(index=[0],data={'zone':'centroid','population':0},geometry=[Point(centroid[0],centroid[1])])
    mesh_tour.index.name = 'index'
    mesh_tour.crs=4326
    if rnodes_file_provided:
        mesh_tour['node_index'] = rnodes.index[0]
elif rnodes_file_provided:
    # use rnodes as mesh.
    print('using road_nodes')
    mesh_tour = population_to_mesh(tourists, mesh=rnodes, step=step_size, col='population', fill_missing='nearest')
else:
    # create a mesh
    #0.01 = 1km 0.005 = 500m
    mesh_tour = population_to_mesh(tourists, step=step_size, col='population', fill_missing='centroid')

38 nodes in multiple zones. will be match to a single zone.
8 unfounded zones
Unfound zones centroid will be added to mesh


In [94]:
mesh_tour.to_file(output_folder + 'tourists_mesh.geojson', driver='GeoJSON', engine=io_engine)

### Catchment

In [95]:
# find TC nodes to mesh distance

In [96]:
if rnodes_file_provided:
    print('using road_nodes')
    node_dist_tour = get_routing_distances(nodes, rnodes, rlinks, mesh_tour, 'length', max_dist)
else:
    node_dist_tour = get_acf_distances(nodes, mesh_tour, crs, max_dist)

### Metrics

In [97]:
res_route_tour = get_catchment('route_id', node_dist=node_dist_tour)

df_route_id['catchment tourists'] = res_route_tour
print(sum([item for key,item in res_route_tour.items()]))

50319.52410738588


In [98]:
res_route_tour = get_catchment('route_type', node_dist=node_dist_tour)

df_route_type['catchment tourists'] = res_route_tour
print(sum([item for key,item in res_route_tour.items()]))

36365.802079383764


# Frequency

In [99]:
# Suppose that headway is the same in both directions : keep the minimum value
idx = df_route_id.groupby(level=0)['headway'].idxmin()
df_route_id = df_route_id.loc[idx]

df_route_id = df_route_id.rename(columns={'headway': 'headway (s)'})
df_route_id = df_route_id.sort_values('route_type', ascending=False)

In [100]:
links['frequency'] = 1/links['headway']

In [101]:
res = (links.groupby('route_id')['frequency'].agg(np.nanmean)*3600).to_dict()

df_route_id['frequency (veh/hours)'] = res
print(np.nansum([item for key,item in res.items()]))

45.78569477963748


In [102]:
res = (links.groupby('route_type')['frequency'].agg(np.nanmean)*3600).to_dict()

df_route_type['frequency (veh/hours)'] = res
print(sum([item for key,item in res.items()]))

14.770594287907802


In [103]:
link = (links.groupby(['route_id','trip_id'])[['frequency']].agg(np.nanmean)*3600)
res = link.reset_index().set_index('route_id')['frequency'].to_dict()
print(np.nansum([item for key,item in res.items()]))

45.78569477963748


In [104]:
link = (links.groupby(['route_type','trip_id'])[['frequency']].agg(np.nanmean)*3600)
res = link.reset_index().set_index('route_type')['frequency'].to_dict()
print(np.nansum([item for key,item in res.items()]))

14.282091733757062


# Operational fleet

In [105]:
def get_fleet(col='route_id'):
    link = links.groupby([col,'trip_id'])[['time','frequency']].agg({'time':np.nansum,'frequency':np.nanmean})
    link['fleet'] = np.ceil(link['frequency'] * link['time'])
    return link.reset_index().groupby(col)['fleet'].agg(np.nansum).to_dict()

In [106]:
res = get_fleet('route_id')

df_route_id['fleet'] = res
print(sum([item for key,item in res.items()]))

52.0


In [107]:
res = get_fleet('route_type')

df_route_type['fleet'] = res
print(sum([item for key,item in res.items()]))

52.0


# Line Length

In [108]:
def get_length(col='route_id'):
    link = links.groupby([col,'trip_id'])[[length_col]].agg(np.nansum)
    if col == 'route_type':
        return link.reset_index().groupby(col)[length_col].agg(np.nansum).to_dict()
    else:
        return link.reset_index().groupby(col)[length_col].agg(np.nanmean).to_dict()

In [109]:
# rpeparation. if legnth is NaN, or if shsape dist travel exist.

length_col = None
if 'length' in links.columns and length_col == None:
    if len(links[links['length'].isnull()])==0:
        length_col = 'length'
        
if 'shape_dist_traveled' in links.columns and length_col == None:
    if len(links[links['shape_dist_traveled'].isnull()])==0:
        length_col = 'shape_dist_traveled'

if length_col == None:
    print('create length from geometry')
    links['length'] = links.to_crs(crs).length
    length_col = 'length'

In [110]:
res = get_length('route_id')

df_route_id['trip length (m)'] = res
print(sum([item for key, item in res.items()]))

730018.0


In [111]:
res = get_length('route_type')

df_route_type['total route length (m)'] = res
print(sum([item for key,item in res.items()]))

1405912


# Stations per line

Type de ligne

In [112]:
dict_nb_trips = links[['route_id', 'trip_id']].drop_duplicates().groupby('route_id')['trip_id'].count().to_dict()
df_route_id['type'] = df_route_id.index.map(dict_nb_trips)
df_route_id['type'] = df_route_id['type'].apply(lambda x: 'circular' if x == 1 else 'linear')

Stops sequence

In [113]:
def get_node_sequence(route_id):
    links_route = links.loc[links.route_id == route_id]
    if df_route_id.loc[route_id]['type'] == 'linear':
        if route_id == 'X Airport':
            trip_id = route_id.replace(' ', '_') + '_0'
        else:
            trip_id = route_id + '_0'
        links_route = links_route.loc[links_route.trip_id == trip_id]
    links_route = links_route.sort_values(by='link_sequence')
    nodes_seq = []
    for i in range(len(links_route)):
        nodes_seq += [links_route.iloc[i]['a']]
    nodes_seq += [links_route.iloc[-1]['b']]
    return nodes_seq

In [114]:
nodes_stops = dict(zip(nodes.index, nodes['stop_name']))

def get_stops_sequence(route_id):
    nodes_seq = get_node_sequence(route_id)
    stops_seq = []
    for node in nodes_seq:
        stops_seq += [nodes_stops[node]]
    return stops_seq

In [115]:
df_route_id['stations sequence'] = [get_stops_sequence(route_id) for route_id in df_route_id.index]
df_route_id['nb stations'] = df_route_id['stations sequence'].apply(lambda x: len(x))

In [116]:
df_route_id.head()

,route_type,veh_capacity (PAX),headway (s),catchment population,catchment jobs,catchment tourists,frequency (veh/hours),fleet,trip length (m),type,stations sequence,nb stations
route_id,,,,,,,,,,,,
IC1,coach,60,82740,0.000000,24.532563,6.536221,0.043510,2.0,220773.5,linear,"[Winter Park parking, Khaybar]",2
IC2,coach,60,82740,0.000000,33.079061,15.251183,0.043510,2.0,43251.5,linear,"[Winter Park parking, Sharaan Reserve]",2
V9,coach,60,1839,1586.413402,3135.866229,6024.330627,1.957586,2.0,8768.0,linear,"[Winter Park parking, Aljadidah South, Almansh...",3
V4,coach,60,2669,0.000000,41.830228,57.013720,1.348820,2.0,34570.0,circular,"[Winter Park parking, Shaden Resort, Sahary Re...",5
V3,coach,60,1655,0.000000,228.847172,216.177699,2.175227,5.0,23205.5,linear,"[Winter Park parking, Habitas Caravan, Maraya,...",5


Number of stations per route_type

In [117]:
stations_route_type = pd.DataFrame(df_route_id.groupby('route_type')['stations sequence'].agg(lambda x: list(set(sum(x, [])))))
stations_route_type['nb stations'] = stations_route_type['stations sequence'].apply(lambda x: len(x))
df_route_type = df_route_type.merge(stations_route_type, left_on=df_route_type.index, right_on=stations_route_type.index, how='left')
df_route_type = df_route_type.rename(columns={'key_0': 'route_type'})
df_route_type = df_route_type.set_index('route_type')

Connexions

In [118]:
iterable = list(zip(nodes['stop_name'], nodes.index))

stops_nodes = defaultdict(set)
for key, value in iterable:
    stops_nodes[key].add(value)
stops_nodes = dict(stops_nodes)

In [119]:
iterable = list(zip(links['a'], links['route_id']))
iterable = iterable + list(zip(links['b'], links['route_id']))

nodes_routes = defaultdict(set)
for key, value in iterable:
    nodes_routes[key].add(value)
nodes_routes = dict(nodes_routes)

In [120]:
stops_routes = {}

for stop, node_list in stops_nodes.items():
    routes = set()
    for node in node_list:
        if node in nodes_routes:
            routes.update(nodes_routes[node])
    stops_routes[stop] = routes

In [121]:
hubs = pd.DataFrame.from_dict(stops_routes, orient='index')
hubs['lines'] = hubs.apply(lambda row: [val for val in row if pd.notnull(val)], axis=1)
hubs = hubs.drop(columns=[i for i in range(len(hubs.columns) - 1)])
hubs['nb_lines'] = hubs['lines'].apply(lambda x: len(x))
hubs = hubs.sort_values(by='nb_lines', ascending=False)

In [122]:
dict_route_type = dict(zip(df_route_id.index, df_route_id['route_type']))
route_order = ['coach', 'bus', 'DRT']

def lines_to_dict(lines):
    route_dict = {route_type: [] for route_type in route_order}
    for line in lines:
        route_type = dict_route_type.get(line)
        if route_type in route_dict:
            route_dict[route_type].append(line)
    route_dict = {k: sorted(v) for k, v in route_dict.items() if v}
    return route_dict

hubs['lines'] = hubs['lines'].apply(lines_to_dict)

In [123]:
from shapely.ops import unary_union

def centroid(geometries):
    combined_geometry = unary_union(geometries)
    return combined_geometry.centroid

centroids = pd.DataFrame(nodes.groupby('stop_name')['geometry'].agg(centroid))
hubs = hubs.merge(centroids, left_on=hubs.index, right_on=centroids.index, how='left')

hubs = hubs.rename(columns={'key_0': 'stop_name'})
hubs = hubs.set_index('stop_name')

In [124]:
hubs['stop_radius'] = hubs['lines'].apply(lambda x: max(catchment_radius[mode] for mode in x.keys()))

In [125]:
# Catchment population, jobs, tourists -- BUT TOO LONG (5 min 30)

# mesh_pop_stops = mesh_pop[mesh_pop['population'] > 0]
# mesh_jobs_stops = mesh_jobs[mesh_jobs['population'] > 0]
# mesh_tour_stops = mesh_tour[mesh_tour['population'] > 0]

# from geopy.distance import geodesic
# def is_within_radius(point, center, radius):
#     distance = geodesic((point.y, point.x), (center.y, center.x)).meters
#     return distance <= radius

# def catchment_pop_stops(row):
#     center = row['geometry']
#     radius = row['stop_radius']
#     population_sum = mesh_pop_stops[mesh_pop_stops.apply(lambda row: is_within_radius(row['geometry'], center, radius), axis=1)]['population'].sum()
#     return population_sum

# def catchment_jobs_stops(row):
#     center = row['geometry']
#     radius = row['stop_radius']
#     jobs_sum = mesh_jobs_stops[mesh_jobs_stops.apply(lambda row: is_within_radius(row['geometry'], center, radius), axis=1)]['population'].sum()
#     return jobs_sum

# def catchment_tour_stops(row):
#     center = row['geometry']
#     radius = row['stop_radius']
#     tourists_sum = mesh_tour_stops[mesh_tour_stops.apply(lambda row: is_within_radius(row['geometry'], center, radius), axis=1)]['population'].sum()
#     return tourists_sum

# hubs['catchment_population'] = hubs.apply(catchment_pop_stops, axis=1)
# hubs['catchment_jobs'] = hubs.apply(catchment_jobs_stops, axis=1)
# hubs['catchment_tourists'] = hubs.apply(catchment_tour_stops, axis=1)

In [126]:
hubs

,lines,nb_lines,geometry,stop_radius
stop_name,,,,
Winter Park parking,"{'coach': ['IC1', 'IC2', 'IC3', 'IC4', 'IC5', ...",13,POINT (37.90674 26.66512),500.0
Old Town South,"{'coach': ['R1', 'R5', 'V1', 'V2', 'X Airport'...",7,POINT (37.91867 26.61641),500.0
Almanshiya,"{'coach': ['R1', 'V2', 'V9'], 'DRT': ['R6']}",4,POINT (37.93949 26.60475),500.0
Daimumah,"{'coach': ['R1', 'R5', 'V2']}",3,POINT (37.92481 26.62847),500.0
Dadan Visitors Center,"{'coach': ['R1', 'R5', 'V2']}",3,POINT (37.91405 26.64345),500.0
...,...,...,...,...
Medina 7,{'DRT': ['R3']},1,POINT (37.90514 26.64698),300.0
Medina 8,{'DRT': ['R3']},1,POINT (37.90534 26.64506),300.0
Medina 10,{'DRT': ['R3']},1,POINT (37.90703 26.64304),300.0


In [127]:
hubs.to_csv(output_folder + 'hubs.csv')

hubs = gpd.GeoDataFrame(hubs, geometry='geometry', crs='EPSG:4326')
hubs.to_file(output_folder + 'hubs.geojson')

In [128]:
def get_connections(row):
    route_id = row.name
    connections = set()
    for station in row['stations sequence']:
        if station in stops_routes:
            connections.update(stops_routes[station])
    connections.discard(route_id)  # Supprimer la route_id de l'ensemble des connexions
    return lines_to_dict(connections), len(connections)

df_route_id[['connexions', 'nb lines connected']] = df_route_id.apply(lambda row: pd.Series(get_connections(row)), axis=1)

In [129]:
df_route_id.head()

,route_type,veh_capacity (PAX),headway (s),catchment population,catchment jobs,catchment tourists,frequency (veh/hours),fleet,trip length (m),type,stations sequence,nb stations,connexions,nb lines connected
route_id,,,,,,,,,,,,,,
IC1,coach,60,82740,0.000000,24.532563,6.536221,0.043510,2.0,220773.5,linear,"[Winter Park parking, Khaybar]",2,"{'coach': ['IC2', 'IC3', 'IC4', 'IC5', 'V1', '...",12
IC2,coach,60,82740,0.000000,33.079061,15.251183,0.043510,2.0,43251.5,linear,"[Winter Park parking, Sharaan Reserve]",2,"{'coach': ['IC1', 'IC3', 'IC4', 'IC5', 'V1', '...",12
V9,coach,60,1839,1586.413402,3135.866229,6024.330627,1.957586,2.0,8768.0,linear,"[Winter Park parking, Aljadidah South, Almansh...",3,"{'coach': ['IC1', 'IC2', 'IC3', 'IC4', 'IC5', ...",14
V4,coach,60,2669,0.000000,41.830228,57.013720,1.348820,2.0,34570.0,circular,"[Winter Park parking, Shaden Resort, Sahary Re...",5,"{'coach': ['IC1', 'IC2', 'IC3', 'IC4', 'IC5', ...",12
V3,coach,60,1655,0.000000,228.847172,216.177699,2.175227,5.0,23205.5,linear,"[Winter Park parking, Habitas Caravan, Maraya,...",5,"{'coach': ['IC1', 'IC2', 'IC3', 'IC4', 'IC5', ...",12


# Vehicle revenue KM 

In [130]:
def get_veh_kmh(col='route_id'):
    link = links.groupby([col,'trip_id'])[[length_col,'frequency']].agg({length_col:np.nansum,'frequency':np.nanmean})
    link['veh_km/h'] = np.ceil(link['frequency'] * link[length_col]) * 3600/1000 #to km/H
    return link.reset_index().groupby(col)['veh_km/h'].agg(np.nansum).to_dict()

In [131]:
res = get_veh_kmh('route_id')

df_route_id['veh.km/h'] = res
print(sum([item for key,item in res.items()]))

817.1999999999999


In [132]:
res = get_veh_kmh('route_type')

df_route_type['veh.km/h'] = res
print(sum([item for key,item in res.items()]))

817.2


# Round trip time

In [133]:
def get_round_trip_time(col='route_id'):
    link = links.groupby([col,'trip_id'])[['time']].agg(np.nansum)
    # return link.reset_index().groupby(col)['time'].agg(np.nansum).to_dict()
    return link.reset_index().groupby(col)['time'].agg(np.nanmean).to_dict()

In [134]:
res = get_round_trip_time('route_id')

df_route_id['trip time (s)'] = res
print(sum([item for key,item in res.items()]))

59811.0


# Average commercial speed

In [135]:
df_route_id['average com. speed (km/h)'] = 3.6 * df_route_id['trip length (m)'] / df_route_id['trip time (s)']

# Add ultimate indicators

In [136]:
# round numbers
for col in ['catchment population', 'catchment jobs', 'catchment tourists', 'trip length (m)', 'trip length (km)', 'veh.km/h', 'trip time (s)', 'average com. speed (km/h)']:
    if col in df_route_id.columns:
        df_route_id[col] = df_route_id[col].apply(lambda x :np.round(x, 0))
    if col in df_route_type.columns:
        df_route_type[col] = df_route_type[col].apply(lambda x :np.round(x, 0))
df_route_id['frequency (veh/hours)'] = df_route_id['frequency (veh/hours)'].apply(lambda x :np.round(x, 2))
df_route_type['frequency (veh/hours)'] = df_route_type['frequency (veh/hours)'].apply(lambda x :np.round(x, 2))

Convert units trip length & trip time

In [137]:
df_route_id['trip length (km)'] = round(df_route_id['trip length (m)'] / 1000, 1)

def sec_to_duree(total_seconds):
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)
    if seconds >= 30:
        minutes += 1
    if minutes == 60:
        hours += 1
        minutes = 0
    time_str = ""
    if hours > 0:
        time_str += f"{hours}h "
    if minutes > 0:  
        time_str += f"{minutes}min "
    return time_str.strip()

df_route_id['one way trip time'] = df_route_id['trip time (s)'].apply(sec_to_duree)
df_route_id['headway'] = df_route_id['headway (s)'].apply(sec_to_duree)

Average distance between stops

In [138]:
df_route_id['average dist. stops (m)'] = df_route_id['trip length (m)'] / (df_route_id['nb stations'] -1)

Annual mileage

In [139]:
df_route_id['annual mileage'] = df_route_id['frequency (veh/hours)'] * 24 * 365 / (df_route_id['fleet'] + df_route_id['fleet'].apply(lambda x: max(1, x / 10))) * df_route_id['trip length (km)']

df_route_id['annual mileage'] = df_route_id.apply(
    lambda row: row['annual mileage'] * 2 if row['type'] == 'linear' else row['annual mileage'], axis=1
)

In [140]:
#df_route_id = df_route_id.fillna('null')
#df_route_type = df_route_type.fillna('null')

# Export dfs to csv

In [141]:
df_route_id = df_route_id.drop(columns=['trip length (m)', 'trip time (s)'])
df_route_id.to_csv(output_folder + 'route_id_metrics.csv')
df_route_id.head()

,route_type,veh_capacity (PAX),headway (s),catchment population,catchment jobs,catchment tourists,frequency (veh/hours),fleet,type,stations sequence,nb stations,connexions,nb lines connected,veh.km/h,average com. speed (km/h),trip length (km),one way trip time,headway,average dist. stops (m),annual mileage
route_id,,,,,,,,,,,,,,,,,,,,
IC1,coach,60,82740,0.0,25.0,7.0,0.04,2.0,linear,"[Winter Park parking, Khaybar]",2,"{'coach': ['IC2', 'IC3', 'IC4', 'IC5', 'V1', '...",12,22.0,73.0,220.8,3h 1min,22h 59min,220774.0,51578.88
IC2,coach,60,82740,0.0,33.0,15.0,0.04,2.0,linear,"[Winter Park parking, Sharaan Reserve]",2,"{'coach': ['IC1', 'IC3', 'IC4', 'IC5', 'V1', '...",12,7.0,52.0,43.3,50min,22h 59min,43252.0,10114.88
V9,coach,60,1839,1586.0,3136.0,6024.0,1.96,2.0,linear,"[Winter Park parking, Aljadidah South, Almansh...",3,"{'coach': ['IC1', 'IC2', 'IC3', 'IC4', 'IC5', ...",14,36.0,20.0,8.8,27min,31min,4384.0,100728.32
V4,coach,60,2669,0.0,42.0,57.0,1.35,2.0,circular,"[Winter Park parking, Shaden Resort, Sahary Re...",5,"{'coach': ['IC1', 'IC2', 'IC3', 'IC4', 'IC5', ...",12,47.0,33.0,34.6,1h 3min,44min,8642.5,136393.20
V3,coach,60,1655,0.0,229.0,216.0,2.18,5.0,linear,"[Winter Park parking, Habitas Caravan, Maraya,...",5,"{'coach': ['IC1', 'IC2', 'IC3', 'IC4', 'IC5', ...",12,104.0,28.0,23.2,50min,28min,5801.5,147681.92


In [142]:
df_route_type.to_csv(output_folder + 'route_type_metrics.csv')
df_route_type

,catchment population,catchment jobs,catchment tourists,frequency (veh/hours),fleet,total route length (m),stations sequence,nb stations,veh.km/h
route_type,,,,,,,,,
DRT,33426.0,23806.0,18264.0,3.30,10.0,50790,"[Old Town South, Khaled AlWalid, Winter Park p...",71,158.0
coach,31341.0,22667.0,13620.0,1.85,40.0,1354146,"[Old Town South, Sadral Gharbaiyah Quarter, Wi...",27,648.0
bus,937.0,1158.0,4481.0,9.63,2.0,976,"[Aljadidah, Aljadidah North Parking]",2,11.0


In [143]:
# TODO : rajouter ligne pour tout le réseau (notamment couverture)

In [144]:
df_route_id = df_route_id.reset_index()
df_route_id = df_route_id[['route_id', 'route_type', 'type', 'trip length (km)', 'nb stations', 'average dist. stops (m)', 'stations sequence', 'connexions', 
                           'headway', 'one way trip time', 'average com. speed (km/h)', 'fleet', 'veh_capacity (PAX)', 'annual mileage',
                           'catchment population', 'catchment jobs', 'catchment tourists']]
df_route_id = df_route_id.rename(
    columns={'route_id': 'Name', 
             'route_type': 'Type',
             'type': 'Route design',
             'headway': 'Headway',
             'trip length (km)': 'Trip length (km)',
             'nb stations': 'Number of stops',
             'average dist. stops (m)': 'Average distance between stops (m)',
             'stations sequence': 'Main stops',
             'one way trip time': 'Travel time',
             'average com. speed (km/h)': 'Average commercial speed (km/h)',
             'veh_capacity (PAX)': 'Vehicle capacity (PAX)',
             'fleet': 'Number of vehicles',
             'annual mileage': 'Annual mileage (km)',
             'catchment population': 'Covered population (inhabitants)',
             'catchment jobs': 'Covered jobs',
             'catchment tourists': 'Covered tourists',
             'connexions': 'Connexion with other lines',
             })

In [145]:
df_route = df_route_id.loc[0]
pd.DataFrame(df_route.T)

,0
Name,IC1
Type,coach
Route design,linear
Trip length (km),220.8
Number of stops,2
Average distance between stops (m),220774.0
Main stops,"[Winter Park parking, Khaybar]"
Connexion with other lines,"{'coach': ['IC2', 'IC3', 'IC4', 'IC5', 'V1', '..."
Headway,22h 59min
Travel time,3h 1min


In [146]:
for i in df_route_id.index:
    route = df_route_id.loc[i, 'Name']
    df_route = df_route_id.loc[i]
    pd.DataFrame(df_route.T).to_csv(output_folder + 'metrics_route_{}.csv'.format(route))

In [ ]:
end_of_notebook

In [147]:
# TODO

## THERMOMETRE LIGNE = PLAN LIGNE DANS LE METRO
 - PRIO : sous-version = plan reseau TC sur fond de carte + 1 carte par ligne
 - Plan du réseau simplifié largeur qui dépend du mode

In [148]:
geom_routes = gpd.GeoDataFrame(links.groupby(['route_id', 'route_color'])['geometry'].agg(unary_union), geometry='geometry', crs='EPSG:4326').reset_index().sort_values(by='route_id')
geom_routes['route_color'] = geom_routes['route_color'].apply(lambda x: ('#'+x).upper())

In [149]:
network = geom_routes.explore(
    column = 'route_id',
    cmap = colors.ListedColormap(geom_routes['route_color'].to_list()),
    tiles = "CartoDB positron",
    style_kwds = {'weight': 3},
    legend_kwds = {'caption': 'Route name'}
    )

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.

In [150]:
network.save(output_folder + 'network.html')

NameError: name 'network' is not defined

In [100]:
for route_id in geom_routes['route_id']:
    geom_route = geom_routes[geom_routes['route_id'] == route_id]
    route_plot = geom_route.explore(
        column = 'route_id',
        cmap = colors.ListedColormap(geom_route['route_color'].to_list()),
        # tiles = "CartoDB positron",
        style_kwds = {'weight': 3},
        legend = False
        )
    route_plot.save(output_folder + 'map_{}.html'.format(route_id))

In [101]:
# import io
# from PIL import Image

# img_data = network._to_png(5)
# img = Image.open(io.BytesIO(img_data))
# img.save(output_folder + 'network.png')

NameError: name 'end_of_notebook' is not defined

# Geomatic outputs : A REVOIR ?
Couverture population par arrêt

In [ ]:
#using get catchment. get the catchment radius of each node (get larger one if used by many mode.)
link = links.groupby('route_type')[['a','b','route_type']].agg({'a':set,'b':set,'route_type':'first'})
link['node'] = link.apply(lambda row: row['a'].union(row['b']), axis=1)
link = link.drop(columns=['a','b'])

# add catchment radius for the route_type
link['catchment_radius'] = link['route_type'].apply(lambda x: catchment_radius.get(x, default_catchment_radius))
link = link.explode('node').reset_index(drop=True)
link = link.sort_values('catchment_radius', ascending=False).drop_duplicates('node', keep='first')
link = node_dist_pop.merge(link, left_on='node_index', right_on='node')
link = link[link['distances'] <= link['catchment_radius']]

In [ ]:
temp_dict = link.groupby('node_index')['population'].sum().to_dict()

In [ ]:
temp_dict

{'node_0': 0.0,
 'node_1': 8566.345274630032,
 'node_10': 62.85056968681102,
 'node_100': 2543.9820090936405,
 'node_101': 2543.9820090936405,
 'node_102': 2543.9820090936405,
 'node_103': 2543.9820090936405,
 'node_104': 1631.2485128411652,
 'node_105': 1631.2485128411652,
 'node_106': 1631.2485128411652,
 'node_107': 1631.2485128411652,
 'node_108': 1469.5686343053208,
 'node_109': 1469.5686343053208,
 'node_11': 1937.5562777693212,
 'node_110': 1469.5686343053208,
 'node_111': 1469.5686343053208,
 'node_112': 1469.5686343053208,
 'node_113': 1469.5686343053208,
 'node_114': 1469.5686343053208,
 'node_119': 62.85056968681102,
 'node_120': 62.85056968681102,
 'node_121': 62.85056968681102,
 'node_122': 62.85056968681102,
 'node_123': 62.85056968681102,
 'node_124': 62.85056968681102,
 'node_125': 1636.246599392545,
 'node_126': 2543.9820090936405,
 'node_127': 1631.2485128411652,
 'node_129': 62.85056968681102,
 'node_130': 62.85056968681102,
 'node_131': 62.85056968681102,
 'node_132

In [ ]:
nodes.head()

AttributeError: 'set' object has no attribute 'head'

In [ ]:
# temp_dict = link.groupby('node_index')['population'].sum().to_dict()
nodes['catchment'] = nodes.index.map(temp_dict.get)

temp_dict = link.groupby('node_index')['catchment_radius'].agg('first').to_dict() 
nodes['catchment_radius'] = nodes.index.map(temp_dict.get)

In [ ]:
nodes.to_file(output_folder + 'nodes.geojson',driver='GeoJSON',engine=io_engine)